# 📐 Word Embeddings: Woorden als getallen

Hoe begrijpt een AI-model de **betekenis** van woorden?

Het antwoord: door elk woord om te zetten in een lijst van getallen — een **vector**. Woorden met een vergelijkbare betekenis krijgen vergelijkbare vectoren.

In dit notebook ontdek je:
1. Hoe word embeddings werken
2. Dat je er rekenkundig mee kunt werken (King - Man + Woman = ?)
3. Hoe je vergelijkbare woorden kunt vinden

---
**Instructie:** Voer iedere cel uit met **Shift+Enter**

⚠️ Het laden van het model (stap 1) duurt ~1-2 minuten bij de eerste keer.

## Stap 1: Pre-trained model laden

We laden een **GloVe model** dat is getraind op miljoenen teksten. Het model kent de vectorrepresentatie van 400.000 Engelse woorden.

In [ ]:
import gensim.downloader as api

# Laad een pre-trained model (400.000 woorden, 50-dimensionale vectoren)
print("Model wordt geladen... (dit duurt ~1-2 minuten)")
model = api.load('glove-wiki-gigaword-50')
print(f"✅ Model geladen! Vocabulaire: {len(model):,} woorden")
print(f"   Elke vector heeft {model.vector_size} dimensies")

## Stap 2: Hoe ziet een word embedding eruit?

Elk woord wordt gerepresenteerd als een rij van 50 getallen. Elk getal staat voor een abstract concept dat het model heeft geleerd.

In [ ]:
import numpy as np

# Bekijk de vector van een woord
woord = "king"
vector = model[woord]

print(f"Vector voor '{woord}':")
print(f"  Lengte: {len(vector)} getallen")
print(f"  Eerste 10 waarden: {np.round(vector[:10], 3)}")
print(f"\nDit is wat het neural network 'ziet' als het het woord '{woord}' leest.")

## Stap 3: Vergelijkbare woorden vinden

Omdat woorden met een vergelijkbare betekenis dicht bij elkaar liggen in de vectorruimte, kunnen we **vergelijkbare woorden** vinden.

In [ ]:
# Zoek woorden die vergelijkbaar zijn met een gegeven woord
zoekwoord = "happy"  # ← Pas dit aan!

vergelijkbaar = model.most_similar(zoekwoord, topn=8)

print(f"Woorden die lijken op '{zoekwoord}':")
print("-" * 40)
for woord, score in vergelijkbaar:
    balk = '█' * int(score * 30)
    print(f"  {woord:15s} {score:.3f}  {balk}")

In [ ]:
# Vergelijk meerdere zoekwoorden
zoekwoorden = ["cat", "dog", "car", "computer", "king"]

for zoek in zoekwoorden:
    top3 = model.most_similar(zoek, topn=3)
    buren = ', '.join([w for w, _ in top3])
    print(f"  {zoek:12s} → {buren}")

## Stap 4: Rekenen met woorden 🧮

Dit is het krachtigste aspect van word embeddings: je kunt er **mee rekenen**!

Het beroemdste voorbeeld:
> **King - Man + Woman = ?**

In [ ]:
# King - Man + Woman = ?
resultaat = model.most_similar(
    positive=['king', 'woman'], 
    negative=['man'], 
    topn=3
)

print("King - Man + Woman = ?")
print("-" * 30)
for woord, score in resultaat:
    print(f"  {woord:12s} ({score:.3f})")

In [ ]:
# Meer voorbeelden van woordrekenkunde
voorbeelden = [
    ("Paris",   "France",  "Germany",  "Parijs - Frankrijk + Duitsland = ?"),
    ("walking", "walk",    "swim",     "Walking - Walk + Swim = ?"),
    ("bigger",  "big",     "small",    "Bigger - Big + Small = ?"),
    ("son",     "man",     "woman",    "Son - Man + Woman = ?"),
]

for pos1, neg, pos2, beschrijving in voorbeelden:
    result = model.most_similar(positive=[pos1, pos2], negative=[neg], topn=1)
    antwoord, score = result[0]
    print(f"  {beschrijving:45s} → {antwoord} ({score:.3f})")

## Stap 5: Visualisatie

Laten we een groep woorden in 2D plotten om te zien hoe het model ze groepeert.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Kies groepen woorden
woord_groepen = {
    'Royalty':  ['king', 'queen', 'prince', 'princess', 'throne'],
    'Dieren':   ['cat', 'dog', 'horse', 'fish', 'bird'],
    'Landen':   ['france', 'germany', 'italy', 'spain', 'japan'],
    'Emoties':  ['happy', 'sad', 'angry', 'love', 'fear'],
}

alle_woorden = []
kleuren = []
kleur_map = {'Royalty': 'royalblue', 'Dieren': 'forestgreen', 
             'Landen': 'coral', 'Emoties': 'mediumpurple'}

for groep, woorden in woord_groepen.items():
    for w in woorden:
        alle_woorden.append(w)
        kleuren.append(kleur_map[groep])

# Haal vectoren op en reduceer naar 2D met PCA
vectoren = [model[w] for w in alle_woorden]
pca = PCA(n_components=2)
coords = pca.fit_transform(vectoren)

# Plot
plt.figure(figsize=(12, 8))
for i, woord in enumerate(alle_woorden):
    plt.scatter(coords[i, 0], coords[i, 1], c=kleuren[i], s=100, zorder=2)
    plt.annotate(woord, (coords[i, 0], coords[i, 1]), 
                fontsize=11, fontweight='bold',
                xytext=(5, 5), textcoords='offset points')

# Legenda
for groep, kleur in kleur_map.items():
    plt.scatter([], [], c=kleur, s=100, label=groep)
plt.legend(fontsize=11, loc='best')

plt.title('Word Embeddings in 2D — Vergelijkbare woorden liggen bij elkaar', fontsize=13)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Stap 6: Welk woord hoort er niet bij?

Omdat het model de betekenis van woorden begrijpt, kan het ook bepalen welk woord **niet in een groep past**.

In [ ]:
# Welk woord hoort er niet bij?
groepen = [
    ["cat", "dog", "horse", "computer"],
    ["paris", "berlin", "london", "banana"],
    ["happy", "sad", "angry", "table"],
    ["doctor", "nurse", "surgeon", "bicycle"],
]

print("Welk woord hoort er niet bij?")
print("-" * 50)
for groep in groepen:
    vreemd = model.doesnt_match(groep)
    print(f"  {groep}  → '{vreemd}'")

## 🧪 Experimenteer zelf!

Probeer:
- Andere woord-analogieën (bijv. `Tokyo - Japan + France = ?`)
- Vergelijkbare woorden zoeken voor een woord naar keuze
- Een eigen "welk woord hoort er niet bij" groep

In [ ]:
# Probeer je eigen woordrekenkunde!
resultaat = model.most_similar(
    positive=['tokyo', 'france'],  # ← Pas dit aan
    negative=['japan'],             # ← Pas dit aan
    topn=3
)

print("Resultaat:")
for woord, score in resultaat:
    print(f"  {woord} ({score:.3f})")